In [7]:
!pip install --user msgpack

  Using cached https://files.pythonhosted.org/packages/8b/fa/975afb5122b563729b93d99b321e3323dee32cdb20b581b5e231d2b49161/msgpack-0.6.1-cp36-cp36m-win_amd64.whl


You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
!pip install --user pymunk
!pip install --user pygame


You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
python setup.py build_ext --inplace

SyntaxError: invalid syntax (<ipython-input-10-2cb21d7d2b68>, line 1)

In [12]:
"""This example spawns (bouncing) balls randomly on a L-shape constructed of 
two segment shapes. Not interactive.
"""

__version__ = "$Id:$"
__docformat__ = "reStructuredText"

# Python imports
import random

# Library imports
import pygame
from pygame.key import *
from pygame.locals import *
from pygame.color import *

# pymunk imports
import pymunk
import pymunk.pygame_util


class BouncyBalls(object):
    """
    This class implements a simple scene in which there is a static platform (made up of a couple of lines)
    that don't move. Balls appear occasionally and drop onto the platform. They bounce around.
    """
    def __init__(self):
        # Space
        self._space = pymunk.Space()
        self._space.gravity = (0.0, -900.0)

        # Physics
        # Time step
        self._dt = 1.0 / 60.0
        # Number of physics steps per screen frame
        self._physics_steps_per_frame = 1

        # pygame
        pygame.init()
        self._screen = pygame.display.set_mode((600, 600))
        self._clock = pygame.time.Clock()

        self._draw_options = pymunk.pygame_util.DrawOptions(self._screen)

        # Static barrier walls (lines) that the balls bounce off of
        self._add_static_scenery()

        # Balls that exist in the world
        self._balls = []

        # Execution control and time until the next ball spawns
        self._running = True
        self._ticks_to_next_ball = 10

    def run(self):
        """
        The main loop of the game.
        :return: None
        """
        # Main loop
        ball_count = 0 
        max_balls = 2000
        while self._running:
            # Progress time forward
            for x in range(self._physics_steps_per_frame):
                self._space.step(self._dt)

            self._process_events()
            if ball_count < max_balls:
                self._update_balls()
                ball_count = ball_count+1
            self._clear_screen()
            self._draw_objects()
            pygame.display.flip()
            # Delay fixed time between frames
            self._clock.tick(50)
            pygame.display.set_caption("fps: " + str(self._clock.get_fps()))

    def _add_static_scenery(self):
        """
        Create the static bodies.
        :return: None
        """
        table_height = 150.0
        line_height = 150.0
        line_width = 400.0
        line_start = 100.0
        bucket_no = 40.0
        peg_gap = 11.0
        peg_h_gap = 11.0
        drop_height = 600.0
        first_fall = 75.0
        peg_rows = 20.0
        bucket_width = line_width / bucket_no
        static_body = self._space.static_body
        static_lines = [pymunk.Segment(static_body, (line_start, table_height), (line_start+line_width, table_height), 1.0),
                        pymunk.Segment(static_body, (line_start, table_height), (line_start, 600.0), 1.0),
                        pymunk.Segment(static_body, (line_start+line_width, table_height), (line_start+line_width, 600.0), 1.0)
                       ]
#                         pymunk.Segment(static_body, (line_start+1.0*bucket_width, table_height), (line_start+1.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+2.0*bucket_width, table_height), (line_start+2.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+3.0*bucket_width, table_height), (line_start+3.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+4.0*bucket_width, table_height), (line_start+4.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+5.0*bucket_width, table_height), (line_start+5.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+6.0*bucket_width, table_height), (line_start+6.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+7.0*bucket_width, table_height), (line_start+7.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+8.0*bucket_width, table_height), (line_start+8.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+9.0*bucket_width, table_height), (line_start+9.0*bucket_width, table_height+line_height), 1.0),
#                         pymunk.Segment(static_body, (line_start+10.0*bucket_width, table_height), (line_start+10.0*bucket_width, 600.0), 1.0)]
#                         pymunk.Circle(static_body, 4.0, offset=(250.0, 450.0)),
#                         pymunk.Circle(static_body, 4.0, offset=(230.0, 400.0)),
#                         pymunk.Circle(static_body, 4.0, offset=(210.0, 400.0)),
#                         pymunk.Circle(static_body, 4.0, offset=(270.0, 400.0)),
#                         pymunk.Circle(static_body, 4.0, offset=(290.0, 400.0)),
#                         pymunk.Circle(static_body, 4.0, offset=(250.0, 350.0))]
        
    
        for bucket in range(int(line_start),int(line_width+line_start), int((line_width+line_start)/bucket_no)):
            static_lines.append(pymunk.Segment(static_body, (bucket, table_height), (bucket, table_height+line_height), 1.0))

        
        for row_count in range(int(peg_rows)):
            for x in range(int(line_start),int(line_start+line_width),int(peg_gap)):
                y = drop_height-first_fall - row_count*peg_h_gap
                if row_count%2 !=0:
                    x+=peg_gap/2
                static_lines.append(pymunk.Circle(static_body, 2.0, offset=(float(x), float(y))))
                    
#         for x in range(int(line_start),int(line_start+line_width),int(peg_gap)):
#             for y in range(int(table_height+200.0),int(drop_height-50.0),int(peg_h_gap)):
#                 if row_count%2 !=0:
#                     x+=peg_gap/2
#                 static_lines.append(pymunk.Circle(static_body, 3.0, offset=(float(x), float(y))))

        for line in static_lines:
            line.elasticity = 0.95
            line.friction = 0.9
        self._space.add(static_lines)

    def _process_events(self):
        """
        Handle game and events like keyboard input. Call once per frame only.
        :return: None
        """
        for event in pygame.event.get():
            if event.type == QUIT:
                self._running = False
            elif event.type == KEYDOWN and event.key == K_ESCAPE:
                self._running = False
            elif event.type == KEYDOWN and event.key == K_p:
                pygame.image.save(self._screen, "bouncing_balls.png")

    def _update_balls(self):
        """
        Create/remove balls as necessary. Call once per frame only.
        :return: None
        """
        self._ticks_to_next_ball -= 1
        if self._ticks_to_next_ball <= 0:
            self._create_ball()
            self._ticks_to_next_ball = 3
    
        # Remove balls that fall below 100 vertically
        balls_to_remove = [ball for ball in self._balls if ball.body.position.y < 100]
        for ball in balls_to_remove:
            self._space.remove(ball, ball.body)
            self._balls.remove(ball)

    def _create_ball(self):
        """
        Create a ball.
        :return:
        """
        mass = 10
        radius = 2.5
        inertia = pymunk.moment_for_circle(mass, 0, radius, (0, 0))
        body = pymunk.Body(mass, inertia)
        x = random.randint(299, 301)
        body.position = x, 600
        shape = pymunk.Circle(body, radius, (0, 0))
        shape.elasticity = 0.65
        shape.friction = 0.35
        self._space.add(body, shape)
        self._balls.append(shape)

    def _clear_screen(self):
        """
        Clears the screen.
        :return: None
        """
        self._screen.fill(THECOLORS["white"])

    def _draw_objects(self):
        """
        Draw the objects.
        :return: None
        """
        self._space.debug_draw(self._draw_options)


if __name__ == '__main__':
    game = BouncyBalls()
    game.run()

From cffi callback <function SpaceDebugDrawOptions.__init__.<locals>.f1 at 0x0000017AB5F90A60>:
Traceback (most recent call last):
  File "D:\Users\hiltss\Github\pymunk\pymunk\space_debug_draw_options.py", line 61, in f1
    @ffi.callback("cpSpaceDebugDrawCircleImpl")
KeyboardInterrupt
